In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import MCXGate
import matplotlib.pyplot as plt

# Helper function: run circuit & plot histogram
def run_and_plot(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    return counts

# Grover’s Algorithm parameters
n = 3
marked = '101'

# Oracle: marks the state |marked>
def grover_oracle(marked):
    n = len(marked)
    qc = QuantumCircuit(n, name='oracle')
    for i, bit in enumerate(reversed(marked)):
        if bit == '0':
            qc.x(i)
    qc.h(n-1)
    mcx_gate = MCXGate(num_ctrl_qubits=n-1)
    qc.append(mcx_gate, list(range(n)))  # replace deprecated mcx
    qc.h(n-1)
    for i, bit in enumerate(reversed(marked)):
        if bit == '0':
            qc.x(i)
    return qc

# Diffuser
def diffuser(n):
    qc = QuantumCircuit(n, name='diffuser')
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    mcx_gate = MCXGate(num_ctrl_qubits=n-1)
    qc.append(mcx_gate, list(range(n)))  # replace deprecated mcx
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    return qc

# Build Grover circuit
qc = QuantumCircuit(n, n)
qc.h(range(n))
qc.append(grover_oracle(marked).to_gate(), range(n))
qc.append(diffuser(n).to_gate(), range(n))
qc.measure(range(n), range(n))

# Draw circuit
print(qc.draw(output="text"))

# Run circuit
counts_g = run_and_plot(qc, shots=2048)
print("Most frequent result:", max(counts_g, key=counts_g.get))


     ┌───┐┌─────────┐┌───────────┐┌─┐      
q_0: ┤ H ├┤0        ├┤0          ├┤M├──────
     ├───┤│         ││           │└╥┘┌─┐   
q_1: ┤ H ├┤1 oracle ├┤1 diffuser ├─╫─┤M├───
     ├───┤│         ││           │ ║ └╥┘┌─┐
q_2: ┤ H ├┤2        ├┤2          ├─╫──╫─┤M├
     └───┘└─────────┘└───────────┘ ║  ║ └╥┘
c: 3/══════════════════════════════╩══╩══╩═
                                   0  1  2 
Most frequent result: 101
